# **Elliptic++ Transactions Dataset**


---
---


Released by: Youssef Elmougy, Ling Liu



School of Computer Science, Georgia Institute of Technology

Contact: yelmougy3@gatech.edu


---

Github Repository: [https://www.github.com/git-disl/EllipticPlusPlus](https://www.github.com/git-disl/EllipticPlusPlus)


If you use our dataset in your work, please cite our paper:





>> Youssef Elmougy and Ling Liu. 2023. Demystifying Fraudulent Transactions and Illicit Nodes in the Bitcoin Network for Financial Forensics.

---



## [SETUP] Import libraries and csv files 

Download dataset from: [https://www.github.com/git-disl/EllipticPlusPlus](https://www.github.com/git-disl/EllipticPlusPlus)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/My\ Drive/Elliptic++\ Dataset/txs_features.csv ./
!cp drive/My\ Drive/Elliptic++\ Dataset/txs_classes.csv ./
!cp drive/My\ Drive/Elliptic++\ Dataset/txs_edgelist.csv ./

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import networkx as nx
#import plotly.graph_objs as go 
#import plotly.offline as py 
import math



In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 

import xgboost as xgb

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 15.8 MB/s eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=138c7b7afc731dc3a39e6bd4e82b7a9fa2f965be699cbe7c19820bc09cf8bfa4
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatib

## Transactions Dataset Overview


---

This section loads the 3 csv files (txs_features, txs_classes, txs_edgelist) and provides a quick overview of the dataset structure and features.

Load saved transactions dataset csv files:

In [3]:
print("\nTransaction features: \n")
df_txs_features = pd.read_csv(r"C:\PC\Linh\MSc_Data_Science\Program\Thesis\Codes\EllipticPlusPlus\EllipticPlusPlus-main\EllipticPlusPlus-main\Transactions Dataset\txs_features.csv")
df_txs_features

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv(r"C:\PC\Linh\MSc_Data_Science\Program\Thesis\Codes\EllipticPlusPlus\EllipticPlusPlus-main\EllipticPlusPlus-main\Transactions Dataset\txs_classes.csv")
df_txs_classes



Transaction features: 


Transaction classes: 



,txId,class
0,3321,3
1,11108,3
2,51816,3
3,68869,2
4,89273,2
...,...,...
203764,158304003,3
203765,158303998,3
203766,158303966,3
203767,161526077,3


Data structure for an example transaction (txId = 272145560):

In [ ]:
print("\ntxs_features.csv for txId = 272145560\n")
df_txs_features[df_txs_features['txId']==272145560]

print("\ntxs_classes.csv for txId = 272145560\n")
df_txs_classes[df_txs_classes['txId']==272145560]

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId=272145560



,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
105573,272145560,24,-0.155493,-0.107012,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.145749,...,2.7732,2.7732,2.7732,2.7732,2.7732,0.001917,2.770883,1.3864,1.3864,2.7728



txs_classes.csv for txId=272145560



,txId,class
105573,272145560,1



txs_edgelist.csv for txId=272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560



Transaction features --- 94 local features, 72 aggregate features, 17 augmented features:


In [ ]:
list(df_txs_features.columns)

['txId',
 'Time step',
 'class',
 'Local_feature_1',
 'Local_feature_2',
 'Local_feature_3',
 'Local_feature_4',
 'Local_feature_5',
 'Local_feature_6',
 'Local_feature_7',
 'Local_feature_8',
 'Local_feature_9',
 'Local_feature_10',
 'Local_feature_11',
 'Local_feature_12',
 'Local_feature_13',
 'Local_feature_14',
 'Local_feature_15',
 'Local_feature_16',
 'Local_feature_17',
 'Local_feature_18',
 'Local_feature_19',
 'Local_feature_20',
 'Local_feature_21',
 'Local_feature_22',
 'Local_feature_23',
 'Local_feature_24',
 'Local_feature_25',
 'Local_feature_26',
 'Local_feature_27',
 'Local_feature_28',
 'Local_feature_29',
 'Local_feature_30',
 'Local_feature_31',
 'Local_feature_32',
 'Local_feature_33',
 'Local_feature_34',
 'Local_feature_35',
 'Local_feature_36',
 'Local_feature_37',
 'Local_feature_38',
 'Local_feature_39',
 'Local_feature_40',
 'Local_feature_41',
 'Local_feature_42',
 'Local_feature_43',
 'Local_feature_44',
 'Local_feature_45',
 'Local_feature_46',
 'Local_fe

## Machine Learning Model Classification


---

This section does data preprocessing, creates the training and testing sets, and runs the Logistic Regression, Random Forest, Multilayer Perceptrons, and XGBoost models as well as the ensembles on the dataset.


Drop transactions without augmented feature values (0.5% not de-anonymized):

In [4]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,0.308900,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932


In [13]:
df_txs_features_merged = pd.merge(df_txs_features, df_txs_classes, on = 'txId')

In [14]:
df_txs_features_merged

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [19]:
df_txs_features_short = df_txs_features_merged[[col for col in df_txs_features_merged.columns if not col.startswith("Aggregate_")]]

In [16]:
df_txs_features_short

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [22]:
df_txs_features_short.columns[95:-1]

Index(['in_txs_degree', 'out_txs_degree', 'total_BTC', 'fees', 'size',
       'num_input_addresses', 'num_output_addresses', 'in_BTC_min',
       'in_BTC_max', 'in_BTC_mean', 'in_BTC_median', 'in_BTC_total',
       'out_BTC_min', 'out_BTC_max', 'out_BTC_mean', 'out_BTC_median',
       'out_BTC_total'],
      dtype='object')

Data transformation on the augmented features using MinMaxScaler:

In [23]:
for column in df_txs_features_short.columns[95:-1]:
    feature = np.array(df_txs_features_short[column]).reshape(-1,1)
    scaler = MinMaxScaler()
    scaler.fit(feature)
    feature_scaled = scaler.transform(feature)
    df_txs_features_short[column] = feature_scaled.reshape(1,-1)[0]

c:\users\link_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\link_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\link_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [24]:
df_txs_features_short

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,0.009891,0.009891,0.009891,0.009891,1.505606e-04,0.009935,0.018790,1.879015e-02,0.009891,3


In [25]:
# remove 'unknown' transactions
data = df_txs_features_short.loc[(df_txs_features_short['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features_short.loc[df_txs_features_short['txId'].isin(data)]
df_txs_features_selected

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,7.022548e-04,2.723834e-04,8.778200e-05,8.171503e-04,6.113009e-04,7.142783e-04,0.001552,1.552291e-03,8.171446e-04,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,7.480472e-02,7.480472e-02,7.480472e-02,7.480472e-02,6.466160e-11,3.647866e-03,0.000022,1.451405e-07,7.480473e-02,2
11,293323,1,-0.172726,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163383,...,3.577994e-06,3.578006e-06,3.579195e-06,3.575573e-06,4.715323e-07,3.511341e-06,0.000007,6.780735e-06,3.569892e-06,2
22,1494462,1,-0.172921,-0.158783,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.163581,...,8.766174e-07,8.766298e-07,8.778192e-07,8.741973e-07,1.442451e-06,6.094506e-07,0.000002,1.632382e-06,8.597370e-07,2
25,1582950,1,-0.169967,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160559,...,4.198289e-05,4.198290e-05,4.198409e-05,4.198047e-05,2.302948e-05,3.817869e-05,0.000080,7.973672e-05,4.197479e-05,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202762,194334585,49,-0.039416,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.026985,...,1.858864e-03,1.858864e-03,1.858865e-03,1.858862e-03,1.992449e-05,1.868443e-03,0.003531,3.531138e-03,1.858833e-03,2
202763,194334621,49,-0.050308,-0.112834,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.038129,...,1.707287e-03,1.707287e-03,1.707288e-03,1.707285e-03,4.973969e-06,1.718449e-03,0.003243,3.243191e-03,1.707255e-03,2
202764,194335206,49,-0.154605,-0.116753,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.144840,...,2.557972e-04,2.557972e-04,2.557984e-04,2.557948e-04,1.933202e-04,2.232165e-04,0.000486,4.858661e-04,2.557661e-04,2
202765,194335216,49,0.708000,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,0.737731,...,1.226060e-02,1.226060e-02,1.226060e-02,1.226060e-02,5.065905e-05,1.233830e-02,0.023291,2.329083e-02,1.226057e-02,2


Split the data into training and testing sets with respect to time steps.

**Training set**: Time steps 1 to 34

**Testing set**: Time steps 35 to 49

In [ ]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

X_data = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_training_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data)]
X_train = X_training_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_testing_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test)]
X_test = X_testing_timesteps.drop(columns=['txId', 'class', 'Time step'])

y_training_timesteps = X_training_timesteps[['class']]
y_training_timesteps = y_training_timesteps['class'].apply(lambda x: 0 if x == 2 else 1 ) # change illicit (class-2) to '0' for classification
y_train = y_training_timesteps

y_testing_timesteps = X_testing_timesteps[['class']]
y_testing_timesteps = y_testing_timesteps['class'].apply(lambda x: 0 if x == 2 else 1 ) # change illicit (class-2) to '0' for classification
y_test = y_testing_timesteps

In [26]:
# change illicit (class-2) to '0' for classification
y = df_txs_features_selected[['class']]
y = y['class'].apply(lambda x: 0 if x == 2 else 1 )

X_train, X_eval, y_train, y_eval = train_test_split(df_txs_features_selected,y,test_size=0.30,random_state=0, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_eval,y_eval,test_size=0.50,random_state=0, shuffle=True)

X_train = X_train.drop(columns=['txId', 'class'])
X_test = X_test.drop(columns=['txId', 'class'])

Run classifiers (LR, RF, MLP, XGB):

In [27]:
# LOGISTIC REGRESSION (LR)
cLR = LogisticRegression(max_iter=1000).fit(X_train.values,y_train.values)
y_preds_LR = cLR.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_LR)

print("Logistic Regression")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_LR, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Logistic Regression
Precision: 0.562 
Recall: 0.131 
F1 Score: 0.213
Micro-Average F1 Score: 0.908


In [28]:
# RANDOM FOREST (RF)
cRF = RandomForestClassifier(n_estimators=50).fit(X_train.values,y_train.values)
y_preds_RF = cRF.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_RF)

print("Random Forest")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_RF, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Random Forest
Precision: 0.987 
Recall: 0.826 
F1 Score: 0.899
Micro-Average F1 Score: 0.982


In [33]:
# MULTILAYER PERCEPTRON (MLP)
cMLP = MLPClassifier(solver='adam', learning_rate_init=0.001, max_iter=200).fit(X_train.values,y_train.values)
y_preds_MLP = cMLP.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_MLP)

print("Multilayer Perceptron (MLP)")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_MLP, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Multilayer Perceptron (MLP)
Precision: 0.850 
Recall: 0.856 
F1 Score: 0.853
Micro-Average F1 Score: 0.972


c:\users\link_\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [31]:
# XGBOOST (XGB)
cXGB = xgb.XGBClassifier(objective="multi:softmax", num_class=2, random_state=42)
cXGB.fit(X_train.values, y_train.values)
y_preds_XGB = cXGB.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_XGB)

print("XGBOOST")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_XGB, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))
#print(confusion_matrix(y, y_pred))

XGBOOST
Precision: 0.974 
Recall: 0.872 
F1 Score: 0.920
Micro-Average F1 Score: 0.986


Run ensemble classifiers (RF+MLP, RF+XGB, MLP+XGB, RF+MLP+XGB):

In [ ]:
#create a dictionary of our models
estimatorsXGBRF=[('RF', cRF), ('XGB', cXGB)]
#create our voting classifier, inputting our models
ensembleXGBRF = VotingClassifier(estimatorsXGBRF, voting='hard')
ensembleXGBRF.fit(X_train.values, y_train.values)
y_preds_XGBRF = ensembleXGBRF.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_XGBRF)

print("Ensemble: XGBoost (XGB) + Random Forest (RF)")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_XGBRF, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Ensemble: XGBoost (XGB) + Random Forest (RF)
Precision: 0.977 
Recall: 0.706 
F1 Score: 0.820
Micro-Average F1 Score: 0.979


In [ ]:
#create a dictionary of our models
estimatorsMLPXGB=[('MLP', cMLP), ('XGB', cXGB)]
#create our voting classifier, inputting our models
ensembleMLPXGB = VotingClassifier(estimatorsMLPXGB, voting='hard')
ensembleMLPXGB.fit(X_train.values, y_train.values)
y_preds_MLPXGB = ensembleMLPXGB.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_MLPXGB)

print("Ensemble: Multilayer Perceptron (MLP) + XGBoost")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_MLPXGB, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Ensemble: Multilayer Perceptron (MLP) + XGBoost
Precision: 0.974 
Recall: 0.596 
F1 Score: 0.739
Micro-Average F1 Score: 0.972


In [ ]:
#create a dictionary of our models
estimatorsRFMLP=[('MLP', cMLP), ('RF', cRF)]
#create our voting classifier, inputting our models
ensembleRFMLP = VotingClassifier(estimatorsRFMLP, voting='hard')
ensembleRFMLP.fit(X_train.values, y_train.values)
y_preds_RFMLP = ensembleRFMLP.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_RFMLP)

print("Ensemble: Random Forest (RF) + Multilayer Perceptron (MLP)")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_RFMLP, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Ensemble: Random Forest (RF) + Multilayer Perceptron (MLP)
Precision: 0.989 
Recall: 0.635 
F1 Score: 0.773
Micro-Average F1 Score: 0.975


In [ ]:
#create a dictionary of our models
estimatorsXGBRFMLP=[('XGB', cXGB), ('MLP', cMLP), ('RF', cRF)]
#create our voting classifier, inputting our models
ensembleXGBRFMLP = VotingClassifier(estimatorsXGBRFMLP, voting='hard')
ensembleXGBRFMLP.fit(X_train.values, y_train.values)
y_preds_XGBRFMLP = ensembleXGBRFMLP.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_XGBRFMLP)

print("Ensemble (all): XGBoost + Random Forest (RF) + Multilayer Perceptron (MLP)")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_XGBRFMLP, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Ensemble (all): XGBoost + Random Forest (RF) + Multilayer Perceptron (MLP)
Precision: 0.962 
Recall: 0.723 
F1 Score: 0.826
Micro-Average F1 Score: 0.980


(LSTM not included)

# **Acknowledgements**


---
---


Released by: Youssef Elmougy, Ling Liu



School of Computer Science, Georgia Institute of Technology

Contact: yelmougy3@gatech.edu


---

Github Repository: [https://www.github.com/git-disl/EllipticPlusPlus](https://www.github.com/git-disl/EllipticPlusPlus)


If you use our dataset in your work, please cite our paper:





>> Youssef Elmougy and Ling Liu. 2023. Demystifying Fraudulent Transactions and Illicit Nodes in the Bitcoin Network for Financial Forensics.

---

